In [ ]:
!pip install MRzeroCore &> /dev/null
!pip install git+https://github.com/imr-framework/pypulseq.git@dev &> /dev/null
!wget https://github.com/MRsources/MRzero-Core/raw/main/documentation/playground_mr0/numerical_brain_cropped.mat &> /dev/null
!wget https://github.com/MRsources/MRzero-Core/raw/main/documentation/playground_mr0/util.py &> /dev/null

(mr0_upload_seq)=
# Upload own seq files
Here  you can upload and simulate own seq files.
Some tested seq-files can be found [here](https://drive.google.com/drive/folders/1EfQRpSXypv3O-t8qn0C6m16xlya_B2Pg). You can download them and then reupload one of these, or an own own seq-file and then simulate them in a brain phantom.


In [ ]:
#@title 1. upload seq file
try:
    # This only runs on google colab
    from google.colab import files
    uploaded = files.upload()
    seq_file = next(iter(uploaded))
except ImportError:
    seq_file = "STE.seq"

In [ ]:
#@title 2. simulate for brain phantom
import numpy as np
# newer numpy versions don't contain this, but pypulseq still relies on it
np.int = int
np.float = float
np.complex = complex

import MRzeroCore as mr0
import pypulseq as pp
import torch
import matplotlib.pyplot as plt
import util

plt.rcParams['figure.figsize'] = [10, 5]
plt.rcParams['figure.dpi'] = 100 # 200 e.g. is really fine, but slower

seq = pp.Sequence()
seq.read(seq_file)

print('load phantom')
# %% S4: SETUP SPIN SYSTEM/object on which we can run the MR sequence external.seq from above
sz = [64, 64]
obj_p = mr0.VoxelGridPhantom.load_mat('numerical_brain_cropped.mat')
brain_phantom_res = 64 #@param {type:"slider", min:16, max:128, step:16}
obj_p = obj_p.interpolate(brain_phantom_res, brain_phantom_res, 1)
obj_p.B0[:] = 0
plot_phantom = False #@param {type:"boolean"}
if plot_phantom: obj_p.plot()

obj_p = obj_p.build()
print('simulate (2D) \n' + seq_file)
seq0 = mr0.Sequence.from_seq_file(seq_file)
# seq0.plot_kspace_trajectory()
# Simulate the sequence
graph = mr0.compute_graph(seq0, obj_p, 200, 1e-3)
signal = mr0.execute_graph(graph, seq0, obj_p)
# PLOT sequence with SIGNAL

sp_adc, t_adc = util.pulseq_plot(seq=seq,signal=signal.numpy())



In [ ]:
signal_part = signal.reshape(32,262,256*2)
print(signal_part.shape)

signal_reordered = torch.zeros(32,262,512,dtype=torch.complex128)
signal_reordered[:,::2,:] = signal_part[:,:131,:]
signal_reordered[:,1::2,:] = signal_part[:,131:,:]
plt.plot(torch.abs(signal_reordered[16,:]))

In [ ]:
kspace = signal_reordered
plt.figure()
plt.imshow(torch.abs(kspace[16]))
image = torch.fft.ifftshift(torch.fft.ifftn(torch.fft.fftshift(kspace)))
plt.figure()
plt.imshow(torch.abs(image[15]).flip(0))
plt.colorbar()

# Neuer Abschnitt

In [ ]:
#@title 3. reconstruct the image
# Unfortunately, we need to limit the resolution as reco_adjoint is very RAM-hungy
print('reconstruct and plot')
seq0.plot_kspace_trajectory()

reco = mr0.reco_adjoint(signal, seq0.get_kspace(), resolution=(64, 64, 1), FOV=(1, 1, 1))
plt.figure()
plt.subplot(121)
plt.title("Magnitude")
plt.imshow(reco[:, :, 0].T.abs(), origin="lower")
plt.colorbar()
plt.subplot(122)
plt.title("Phase")
plt.imshow(reco[:, :, 0].T.angle(), origin="lower", vmin=-np.pi, vmax=np.pi)
plt.colorbar()
plt.show()